In the past two notebooks, we have simulated parametric data, and calculated emperical power for these simulations. Here, we'll compare the performance of the distribution-based power, empercial power and power calculated using the curve fitting parameter.

In [1]:
import os
import pickle
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn

import emp_power.traditional as trad
import emp_power.effects as eff
import emp_power.plot as plot
import emp_power.summarize as summarize

% matplotlib inline
sn.set_style('ticks')

In [2]:
warnings.filterwarnings('ignore')

We've performed 100 simulations, which are stored in the simulation directory.

In [3]:
num_rounds = 100

sim_location = './simulations'
if not os.path.exists(sim_location):
    raise ValueError('The power simulations do not exist.'
                     'Please go back to notebooks 2 and 3 and'
                     'calculate power.'
                     )
return_dir = os.path.join(sim_location, 'extrapolation')
if not os.path.exists(return_dir):
    os.makedirs(return_dir)

We'll fit our power curves using a power value of 0.05, and calculate the fitted curves using the same value. This was the critical value used to calculate distribution-based and emperical power.

In [4]:
alpha = 0.05

Power in previous notebooks was calculated with between 5 and 95 observations, with 10 observation steps.

In [5]:
counts = np.arange(5, 100, 10)

We'll use the spectral colormap, scaled by the count depth.

In [6]:
colormap = 'Spectral'

colors = {count: list(sn.color_palette(colormap, n_colors=len(counts))[i])
          for (i, count) in enumerate(counts)}
# check_color = {count: list(colors[i]) for i, count in enumerate(counts)}

We'll look at effect sizes calculated based on the f, t, and z distributions.

In [7]:
effect_types = ['f', 't', 'z']

We'll compare the behavior of distribution-based power, emperical power and the power calculated from curve fitting on the parametric tests. We can compare the behavior of emperical power, and power fit to emperical values for all tests.

In [8]:
parametric_tests = ['ttest_1', 'ttest_ind', 'anova_3', 'anova_8', 'correlation']
all_tests = ['ttest_1', 'ttest_ind', 'anova_3', 'anova_8', 'permanova', 'correlation', 'mantel']

Finally, we'll create a set of parameters for each of the distributions being evaluated. This includes the clean name, which provides a prettier title in plots, the number of groups in the tested (2 for everything except the ANOVAs), and the location of both the input emperical power calculations and output summary tables.

In [9]:
distributions = {'ttest_1': {'clean_name': 'One Sample\nT Test',
                               'num_groups': 2,
                               'input_dir': './simulations/power/ttest_1/',
                               'return_fp': './simulations/extrapolation/ttest_1.txt'
                               },
                 'ttest_ind': {'clean_name': 'Independent Sample\n T Test',
                               'num_groups': 2,
                               'input_dir': './simulations/power/ttest_ind',
                               'return_fp': './simulations/extrapolation/ttest_ind.txt'
                               },
                 'anova_3': {'clean_name': 'One way ANOVA\n(3 groups)',
                             'num_groups': 3,
                             'input_dir': './simulations/power/anova_3',
                             'return_fp': './simulations/extrapolation/anova_3.txt'
                             },
                 'anova_8': {'clean_name': 'One way ANOVA\n(8 groups)',
                             'num_groups': 8,
                             'input_dir': './simulations/power/anova_8',
                             'return_fp': './simulations/extrapolation/anova_8.txt'
                             },
                 'correlation': {'clean_name': 'Correlation',
                                 'num_groups': 2,
                                 'input_dir': './simulations/power/correlation',
                                 'return_fp': './simulations/extrapolation/correlation.txt'
                                 },
                 'permanova': {'clean_name': 'PERMANOVA',
                               'num_groups': 2,
                               'input_dir': './simulations/power/permanova/',
                               'return_fp': './simulations/extrapolation/permanova.txt'
                               },
                 'mantel': {'clean_name': 'Mantel',
                            'num_groups': 2,
                            'input_dir': './simulations/power/mantel/',
                            'return_fp': './simulations/extrapolation/mantel.txt'
                            },
                 }

We'll start by calculating the emperical and extrapolated effect sizes for the parametric tests.

In [10]:
for test_name in all_tests:
    power_dir = distributions[test_name]['input_dir']
    num_groups = distributions[test_name]['num_groups']
    return_fp = distributions[test_name]['return_fp']
    
    if not os.path.exists(power_dir):
        raise ValueError('%s does not exist' % power_dir)
        
    summaries = []
    
    # Loops through the rounds
    for i in range(num_rounds):
        # Loads through the power simulation for the round
        power_fp = os.path.join(power_dir, 'simulation_%i.p' % i)
        
        with open(power_fp, 'rb') as f_:
            sim = pickle.load(f_)
        if 'alpha' not in sim:
            sim['alpha'] = 0.05
        
        # Pulls the previously calculated power values
        run_summary = summarize.summarize_power(power_summary=sim,
                                                sim_num=i,
                                                test=test_name,
                                                colors=colors,
                                                dists=effect_types,
                                                num_groups=num_groups,
                                               )
        summaries.append(pd.DataFrame(run_summary))
    summaries = pd.concat(summaries)
    summaries.to_csv(return_fp, sep='\t')
    distributions[test_name]['summary'] = summaries


In [11]:
sim

{'alpha': 0.05,
 'counts': array([ 5, 15, 25, 35, 45, 55, 65, 75]),
 'depth': 99,
 'emperical_power': array([[ 0.27,  0.84,  0.97,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ],
        [ 0.16,  0.86,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ],
        [ 0.18,  0.88,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ],
        [ 0.27,  0.88,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ],
        [ 0.25,  0.88,  0.99,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ]]),
 'num_obs': 88,
 'original_p': 0.01,
 'traditional_power': None}

In [ ]:
counts = sim['counts']
emperical = sim['emperical_power']
(empr_r, empr_c) = emperical.shape

In [ ]:
sim['emperical_power'].shape

We're going to plot a comparison between the emperical and traditional power calculations, to see if there's a strong relationship. We'll plot the distribution-based power on the x axis and the emperical power on the y axis.

In [ ]:
# Sets up the figure and axes
tr_fig, tr_axes = plt.subplots(4, 5)
tr_fig.set_size_inches(10, 8)

# Hides the axes until they're needed
for ax_r in tr_axes[1:]:
    for ax in ax_r:
        ax.set_visible(False)

sn.despine()

for idx, (test_name, ax) in enumerate(zip(*(parametric, tr_axes[0]))):
    summary = distributions[test_name]['summary']
    plot.gradient_regression(ax, y='emperical', 
                             x='traditional', 
                             gradient='colors', 
                             data=summary, 
                             alpha=0.25)
    plot.format_regression_axis(ax)
    ax.set_title(distributions[test_name]['clean_name'])
    ax.set_xticklabels(ax.get_xticks())
tr_axes[0][0].set_yticklabels(ax.get_yticks())
tr_axes[0][0].set_ylabel('Emperical Power')
tr_axes[0][2].set_xlabel('Traditional Power')

We can also look at the fit data behaves. So, we'll add an additional three rows to the plot, which show the 

In [ ]:
for ax_r in tr_axes[1:]:
    for ax in ax_r:
        ax.set_visible(True)
for ax in tr_axes[0]:
    ax.set_xticklabels('')
    ax.set_xlabel('')
    
for idc, test_name in enumerate(tests):
    summary = distributions[test_name]['summary']
    for metric, ax_reg in zip(*[extrapolated, tr_axes.T[idc][1:]]):
        plot.gradient_regression(ax=ax_reg, 
                            x='traditional', 
                            y='%s_power' % metric, 
                            gradient='colors', 
                            alpha=0.25,
                            data=summary
                            )
        plot.format_regression_axis(ax_reg)
        if metric == 'z':
            ax_reg.set_xticklabels(ax_reg.get_xticks())
        if test_name == tests[0]:
            ax_reg.set_yticklabels(ax_reg.get_yticks())
            ax_reg.set_ylabel('%s Power' % metric.replace('_', ' ').capitalize())
        
tr_axes[-1][2].set_xlabel('Distribution-based Power')
tr_fig

For most of the effect sizes and fits, we find the behavior of the curve 

Let's also compare the behavior of the emperical power and the fit power curves. 

In [ ]:
# Sets up the figure and axes
er_fig, er_axes = plt.subplots(3, 5)
er_fig.set_size_inches(10, 6)
sn.despine()

for idc, test_name in enumerate(tests):
    summary = distributions[test_name]['summary']
    for metric, ax_reg in zip(*[extrapolated, er_axes.T[idc]]):
        plot.gradient_regression(ax=ax_reg, 
                            x='emperical', 
                            y='%s_power' % metric, 
                            gradient='colors', 
                            alpha=0.25,
                            data=summary
                            )
        plot.format_regression_axis(ax_reg)
        if metric == 'z_power':
            ax_reg.set_xticklabels(ax_reg.get_xticks())
        if test_name == tests[0]:
            ax_reg.set_yticklabels(ax_reg.get_yticks())
            ax_reg.set_ylabel('%s Power' % metric.replace('_', ' ').capitalize())
        
er_axes[-1][2].set_xlabel('Emperical Power')

Based on the color by depth, it looks like the emperical power and extrapolated values do not behave as well at lower count values. We can recalculate the fit curves excluding these lower values, to see if this improves the fit. We'll exclude points with less tahn 5 observations and emperical values between less than 0.05 or greater than 0.95, as the percentage point functions tend towards aspmototic values at thoses extremes.

In [ ]:
# Sets up the figure and axes
em_fig, em_axes = plt.subplots(3, 5)
em_fig.set_size_inches(10, 6)

sn.despine()

for idx, (test_name, ax) in enumerate(zip(*(tests, em_axes[0]))):
    summary = distributions[test_name]['summary']
    drop_index = summary.index[((summary['counts'] == 5) | 
                                (summary['emperical'] < 0.05) | 
                                (summary['emperical'] > 0.95))]
    mod = summarize.modify_effect_size(summary, drop_index, extrapolated)
    mod.loc[mod['counts'] == 5, 'emperical'] = np.nan
    for metric, ax_reg in zip(*[extrapolated, em_axes.T[idx]]):
        plot.gradient_regression(ax=ax_reg, 
                                 x='emperical', 
                                 y='%s_power' % metric, 
                                 gradient='colors', 
                                 alpha=0.25,
                                 data=mod
                                 )
        plot.format_regression_axis(ax_reg)
        if metric == 'z_power':
            ax_reg.set_xticklabels(ax_reg.get_xticks())
        if test_name == tests[0]:
            ax_reg.set_yticklabels(ax_reg.get_yticks())
            ax_reg.set_ylabel(
                '%s Power' % metric.replace('_', ' ').capitalize()
                )

The exclusion of the first set of emperical points appears to slightly improve the fit of the dataset. However, for smaller datasets, it may be worth the uncertainty assoicated with the extreme values, as long as th majority of values are not extreme.

We also notice that the F distribution with $k - 1$ degrees of freedom does not behave well for the 8 group ANOVA. With fewer groups, the F distribution behaves simillarly to the T and Z distribution. However, as the number of groups increase, the distribution does not behave appropriately. This suggests that the power calculation may be better approximated by assuming a normal distribution than using the approximation based on group size.

Finally, an additional consideration in selecting the appropriate effect size metric is to determine how frequently a particular metric cannot be defined.

In [ ]:
all_tests = pd.concat([distributions[test_name]['summary'] for test_name in tests])
all_tests.groupby('test').count()[['f_effect', 't_effect', 'z_effect']] / 3000

So, in this notebook we have demonstrated that emperically calculated power behaves as well as distribution-based calculations for parametric distributions. We've furthermore observed that fitting the emperical curves with effect size parameters generates power curves which closely fit both the distribution and emperical data. However, we find the performance o the different fitting approaches varies. 

The calculation based on the z distribution is the most robust, while the calculation based on the F distribution is the least robust, where robustness is measured as the percentgae of calculations where power could not be calculated.

In the next notebook, we will explore the behavior of the permutative tests under similar condtions, to validate that these tests also work.